In [ ]:
# RESEARCH PAPER (https://arxiv.org/abs/1504.06375)
# https://github.com/lc82111/Keras_HED
# Tensorflow work here, https://github.com/harsimrat-eyeem/holy-edge

In [1]:
# all imports
from skimage.filters import threshold_local
import numpy as np
import h5py
import argparse
import cv2,imutils
from matplotlib import pyplot as plt
import numpy as np
import time as t
# https://blog.dominodatalab.com/interactive-dashboards-in-jupyter/
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import widgets
from pylab import rcParams
from PIL import Image
from glob import glob
from statistics import mode
import os,pytesseract,editdistance,re
rcParams['figure.figsize'] = 7, 7
%matplotlib inline

In [2]:
# %load ../../src/img_utils.py
# %load img_utils.py

def showimg(img):
    plt.imshow(img,cmap='gray')
    plt.show()

def resizeAndPad(img, size, padColor=0):
    '''
        Aspect Ratio Rescaling of image.
        Throws error if height / width is not generatable.
        Adds black padding to heigh / width appropriately.
    '''
    h, w = img.shape[:2]
    sh, sw = size
    # interpolation method
    if h > sh or w > sw: # shrinking image
        interp = cv2.INTER_AREA
    else: # stretching image
        interp = cv2.INTER_CUBIC

    # aspect ratio of image
    aspect = w/h

    # compute scaling and pad sizing
    if aspect > 1: # horizontal image
        new_w = sw
        new_h = np.round(new_w/aspect).astype(int)
        pad_vert = (sh-new_h)/2
        pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
        pad_left, pad_right = 0, 0
    elif aspect < 1: # vertical image
        new_h = sh
        new_w = np.round(new_h*aspect).astype(int)
        pad_horz = (sw-new_w)/2
        pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
        pad_top, pad_bot = 0, 0
    else: # square image
        new_h, new_w = sh, sw
        pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

    # set pad color
    if len(img.shape) is 3 and not isinstance(padColor, (list, tuple, np.ndarray)): # color image but only one color provided
        padColor = [padColor]*3

    # scale and pad
    scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
    scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

    return scaled_img

def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
 
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
 
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
 
	# return the edged image
	return edged

In [3]:
def get_min_max_value(img,h=512,w=512,ch=3):
    max_val=-1;min_val=255
    if ch==3:
        for i  in range(h):
            for j in range(w):
                if img[i][j][0] > max_val:
                    max_val=img[i][j][0]
                if img[i][j][1] > max_val:
                    max_val = img[i][j][1]
                if img[i][j][2] > max_val:
                    max_val = img[i][j][2]
                if img[i][j][0] < min_val:
                    min_val=img[i][j][0]
                if img[i][j][1] < min_val:
                    min_val = img[i][j][1]
                if img[i][j][2] < min_val:
                    min_val = img[i][j][2]
    elif ch==1:
        for i  in range(h):
            for j in range(w):
                if img[i][j][0] > max_val:
                    max_val=img[i][j][0]
                if img[i][j][0] < min_val:
                    min_val=img[i][j][0]
    return min_val,max_val

In [4]:
from __future__ import print_function
import os
from keras.utils import plot_model
from keras import backend as K
from keras import callbacks
import numpy as np
# import pdb
# from src.utils.HED_data_parser import DataParser
from src.networks.hed import hed
from src.utils.HED_data_parser2 import DataParser2
from src.utils.HED_test_parser import TestParser
from src.networks.hed2 import hed2

Using TensorFlow backend.


In [5]:
def generate_minibatches(dataParser, train=True):
    #pdb.set_trace()
    while True:
        if train:
            batch_ids = np.random.choice(dataParser.training_ids, dataParser.batch_size_train)
        else:
            batch_ids = np.random.choice(dataParser.validation_ids, dataParser.batch_size_train*2)
        ims, ems, _ = dataParser.get_batch(batch_ids)
        yield(ims, [ems, ems, ems, ems, ems, ems])
        
def generate_testbatches(dataParser):
    while True:
        batch_ids = np.random.choice(dataParser.all_ids,20)
        ims, ems, _ = dataParser.get_batch(batch_ids)
        yield(ims, [ems, ems, ems, ems, ems, ems])

### Train on HED-BSDS Dataset

In [ ]:
# model_name = 'HEDSeg3'
# model_dir     = os.path.join('checkpoints', model_name)
# csv_fn        = os.path.join(model_dir, 'train_log.csv')
# checkpoint_fn = os.path.join(model_dir, 'checkpoint.{epoch:02d}-{val_loss:.2f}.hdf5')

# batch_size_train = 10

# # environment
# K.set_image_data_format('channels_last')
# K.image_data_format()
# os.environ["CUDA_VISIBLE_DEVICES"]= '0'
# if not os.path.isdir(model_dir): os.makedirs(model_dir)

# # prepare data
# input_shape=(512,512,3)
# dataParser = DataParser2(batch_size_train,
#                          train_data_dir='/home/ydalal/rasidrev/src_other_dl_projects/Keras_HED-master/',
#                          img_mask_pair_file='train_pair.lst',
#                          input_shape=input_shape,
#                          train_split=0.8,
#                          isAbsolutePath=True
#                         )

# # model
# model = hed2(input_shape=input_shape,weights=None)
# plot_model(model, to_file=os.path.join(model_dir, 'model.pdf'), show_shapes=True)

# model.load_weights('./checkpoints/BillSeg2/checkpoint.70-0.18.hdf5')

# # Training
# # call backs
# checkpointer = callbacks.ModelCheckpoint(filepath=checkpoint_fn, 
#                                          verbose=2, 
#                                          save_best_only=True) 
# # Make it True if space issue

# csv_logger  = callbacks.CSVLogger(csv_fn, append=True, separator=';')

# tensorboard = callbacks.TensorBoard(log_dir=model_dir, 
#                                     histogram_freq=0, 
#                                     batch_size=10,
#                                     write_graph=False,
#                                     write_grads=True,
#                                     write_images=False)
# # Set it to false if space issue

# epochs=1024
# train_history = model.fit_generator(
#                     generate_minibatches(dataParser,train=True),
#                     # max_q_size=40, workers=1,
#                     steps_per_epoch=dataParser.steps_per_epoch,  #batch size
#                     epochs=epochs,
#                     validation_data=generate_minibatches(dataParser, train=True),
#                     validation_steps=dataParser.validation_steps,
#                     callbacks=[checkpointer, csv_logger, tensorboard])

# test_generator=generate_minibatches(dataParser,train=False)
# ims,_=next(test_generator)
# predictions=model.predict(ims,batch_size=1,verbose=1,steps=None)

# for i in range(min(2,len(ims))):
#     img=ims[i]
#     min_val,max_val=get_min_max_value(img)
#     img=(img-min_val)/255
#     plt.imshow(img);plt.show()
#     showimg(predictions[5][i].squeeze())
# #     for prediction in predictions:
# #         showimg(prediction[i].squeeze())

### Train on Bill Data

In [6]:
model_name = 'model'
model_dir     = os.path.join('/media/kartik/New Volume/model/numberplate/', model_name)
csv_fn        = os.path.join(model_dir, 'train_log.csv')
checkpoint_fn = os.path.join(model_dir, 'checkpoint.{epoch:02d}-{val_loss:.2f}.hdf5')

In [ ]:
import cv2 , os
import imutils
from glob import glob

In [ ]:
inputPath = '/home/kartik/rasidrev/resources/new/img'
#test_file = open()
i = 0
for root, dirs, files in os.walk(inputPath, topdown=False):
    img_path = root
    root.split('/')[-1] = 'mask'
    mask_path = root
    print(root.split('/')[-1] )
    for root, dirs, files in os.walk(root, topdown=False):
        for f in files:
            if i == 1 :
                print(root+f)

In [7]:
from glob import glob
import os
n = 'bhavuk'
combined_labels_file=open('/home/kartik/rasidrev/resources/free_flow/label.txt','w')
counter = 0

for fpath in glob('/home/kartik/Downloads/totaltext/Images/Train/new/*.jpg')+glob('/home/kartik/Downloads/totaltext/Images/Train/new/*.png'):
    imgpath=os.path.abspath(fpath)
    maskpath='/'.join(imgpath.split('/')[:-3])+'/train_mask/new/'+imgpath.split('/')[-1]
    #print(maskpath)
    combined_labels_file.write(imgpath+' '+maskpath+'\n')
    counter = counter +1 
combined_labels_file.close()

In [ ]:
txt_list = []
for txt in glob('/home/kartik/rasidrev/resources/label_txt/*.txt'):
    txt_list.append(txt)
txt_list

In [ ]:
# Write labels info files into a main file for training
counter = 0
combined_labels_file=open('/home/kartik/rasidrev/resources/all_bills_labels.txt','w')
for fname in txt_list:
    reader=open(fname,'r')
    for line in reader:
        counter = counter + 1
        combined_labels_file.write(line)
    reader.close()
combined_labels_file.close()

In [17]:
batch_size_train = 3

# environment
K.set_image_data_format('channels_last')
K.image_data_format()
os.environ["CUDA_VISIBLE_DEVICES"]= '0'
if not os.path.isdir(model_dir): os.makedirs(model_dir)

# prepare data
input_shape=(1024,1024,3)
dataParser = DataParser2(batch_size_train,
#                         train_data_dir='/home/ydalal/rasidrev/resources/train_data_printed_bills/',
                         train_data_dir='/home/kartik/rasidrev/resources/free_flow/',
                         img_mask_pair_file='label.txt',
                         input_shape=input_shape,
                         train_split=0.81,
                         isAbsolutePath=True
                        )

assertion1 :  297 3
assertion2 :  70 70


In [18]:
# model, 
# Current model trains on 480 480 3 input picture
model = hed2(input_shape=(1024,1024,3),weights=None)

In [ ]:
# if weights are not loaded during initialization, try loading them now
# model.load_weights('./checkpoints/HEDSeg2/checkpoint.02-0.13.hdf5')
# model.load_weights('./checkpoints/BillSeg/checkpoint.10-0.04.hdf5')
model.load_weights('/media/kartik/New Volume/model/freeflow/model/mag/model/model/model/model/checkpoint.743-0.06.hdf5')
# model.load_weights('./checkpoints/BillSeg_2/checkpoint.70-0.18.hdf5')
# model.load_weights('./checkpoints/BillPredictor/checkpoint.43-0.18.hdf5')
# model.load_weights('./checkpoints/BillSeg_4/checkpoint.27-0.13.hdf5')

In [19]:
model.load_weights('/media/kartik/New Volume/model/numberplate/model/checkpoint.03-0.12.hdf5')

In [20]:
# Training
# call backs
checkpointer = callbacks.ModelCheckpoint(filepath=checkpoint_fn, 
                                         verbose=2, 
                                         save_best_only=True) 
# Make it True if space issue

csv_logger  = callbacks.CSVLogger(csv_fn, append=True, separator=';')

tensorboard = callbacks.TensorBoard(log_dir=model_dir, 
                                    histogram_freq=0, 
                                    batch_size=2,
                                    write_graph=False,
                                    write_grads=True,
                                    write_images=True)
# Set it to false if space issue

In [ ]:
# https://keras.io/preprocessing/image/

In [ ]:
# # we create two instances with the same arguments
# data_gen_args = dict(featurewise_center=True,
#                      featurewise_std_normalization=True,
#                      rotation_range=90.,
#                      width_shift_range=0.1,
#                      height_shift_range=0.1,
#                      zoom_range=0.2)
# image_datagen = ImageDataGenerator(**data_gen_args)
# mask_datagen = ImageDataGenerator(**data_gen_args)

# # Provide the same seed and keyword arguments to the fit and flow methods
# seed = 1
# # image_datagen.fit(images, augment=True, seed=seed)
# # mask_datagen.fit(masks, augment=True, seed=seed)
# image_generator = image_datagen.flow_from_directory(
#     '../../resources/train_mask/img',
#     target_size=(512,512),
#     class_mode=None,
#     seed=seed)

# mask_generator = mask_datagen.flow_from_directory(
#     '../../resources/train_mask/mask',
#     target_size=(512,512),
#     class_mode=None,
#     seed=seed)

# # combine generators into one which yields image and masks
# train_generator = zip(image_generator, mask_generator)

In [ ]:
# epochs=1024
# train_history = model.fit_generator(
#                     train_generator,
#                     # max_q_size=40, workers=1,
#                     steps_per_epoch=dataParser.steps_per_epoch,  #batch size
#                 callbacks=[checkpointer, csv_logger, tensorboard])         epochs=epochs)
    
# #     validation_data=generate_minibatches(dataParser, train=True),
# #                     validation_steps=dataParser.validation_steps,
# #                

In [ ]:
epochs=1024
train_history = model.fit_generator(
                    generate_minibatches(dataParser,train=True),
                    # max_q_size=40, workers=1,
                    steps_per_epoch=dataParser.steps_per_epoch,  #batch size
                    epochs=epochs,
                    validation_data=generate_minibatches(dataParser, train=True),
                    validation_steps=dataParser.validation_steps,
                    callbacks=[checkpointer, csv_logger, tensorboard])

Epoch 1/1024
99/99 [==============================] - 132s 1s/step - loss: 0.2155 - o1_loss: 0.0806 - o2_loss: 0.0592 - o3_loss: 0.0322 - o4_loss: 0.0159 - o5_loss: 0.0159 - ofuse_loss: 0.0116 - ofuse_ofuse_pixel_error: 0.0332 - val_loss: 0.1904 - val_o1_loss: 0.0788 - val_o2_loss: 0.0490 - val_o3_loss: 0.0262 - val_o4_loss: 0.0104 - val_o5_loss: 0.0172 - val_ofuse_loss: 0.0088 - val_ofuse_ofuse_pixel_error: 0.0285
Epoch 2/1024
99/99 [==============================] - 128s 1s/step - loss: 0.2126 - o1_loss: 0.0884 - o2_loss: 0.0643 - o3_loss: 0.0309 - o4_loss: 0.0112 - o5_loss: 0.0106 - ofuse_loss: 0.0072 - ofuse_ofuse_pixel_error: 0.0251 - val_loss: 0.2103 - val_o1_loss: 0.0991 - val_o2_loss: 0.0736 - val_o3_loss: 0.0246 - val_o4_loss: 0.0052 - val_o5_loss: 0.0041 - val_ofuse_loss: 0.0038 - val_ofuse_ofuse_pixel_error: 0.0157
Epoch 3/1024
99/99 [==============================] - 129s 1s/step - loss: 0.1952 - o1_loss: 0.0821 - o2_loss: 0.0616 - o3_loss: 0.0290 - o4_loss: 0.0089 - o5_los

In [ ]:
epochs=1024
train_history = model.fit_generator(
    
                    generate_minibatches(dataParser,train=True),
                    # max_q_size=40, workers=1,
                    steps_per_epoch=dataParser.steps_per_epoch,  #batch size
                    epochs=epochs,
                    validation_data=generate_minibatches(dataParser, train=True),
                    validation_steps=dataParser.validation_steps,
                    callbacks=[checkpointer, csv_logger, tensorboard])

In [ ]:
epochs=1024
train_history = model.fit_generator(
                    generate_minibatches(dataParser,train=True),
                    # max_q_size=40, workers=1,
                    steps_per_epoch=dataParser.steps_per_epoch,  #batch size
                    epochs=epochs,
                    validation_data=generate_minibatches(dataParser, train=True),
                    validation_steps=dataParser.validation_steps,
                    callbacks=[checkpointer, csv_logger, tensorboard])

In [ ]:
# First Model
epochs=1024
train_history = model.fit_generator(
                    generate_minibatches(dataParser,train=True),
                    # max_q_size=40, workers=1,
                    steps_per_epoch=dataParser.steps_per_epoch,  #batch size
                    epochs=epochs,
                    validation_data=generate_minibatches(dataParser, train=True),
                    validation_steps=dataParser.validation_steps,
                    callbacks=[checkpointer, csv_logger, tensorboard])

In [ ]:
test_generator=generate_minibatches(dataParser,train=False)
ims,_=next(test_generator)
predictions=model.predict(ims,batch_size=1,verbose=1,steps=None)

#### Test Data Only

In [ ]:
test_file=open('./testfile.txt','w')
from glob import glob
import os
for fpath in glob('l/img/*.png'):
    print(fpath)
    test_file.write(os.path.abspath(fpath)+' \n')
test_file.close()

In [ ]:
input_shape=(1024,1024,3)
testParser=TestParser(img_mask_pair_file='./testfile.txt',
                      input_shape=input_shape)

In [ ]:
!head -n5 testfile.txt

In [ ]:
test_generator=generate_testbatches(testParser)
ims,_=next(test_generator)
predictions=model.predict(ims,batch_size=1,verbose=1,steps=None)

In [ ]:
for img in ims:
    print(img.shape)

In [ ]:
for i in range(min(20,len(ims))):
    img=ims[i]
    min_val,max_val=get_min_max_value(img)
    img=(img-min_val)/255
    cv2.imwrite(str(i)+'img.png',(img*255))
    plt.imshow(img);plt.show()
    showimg(predictions[5][i].squeeze())
    cv2.imwrite(str(i)+'prediction.png',(predictions[5][i].squeeze()*255))

### Load Unseen Images

In [ ]:
# prepare data
input_shape=(512,512,3)
dataParser = DataParser2(batch_size_train,
#                         train_data_dir='/home/ydalal/rasidrev/resources/train_data_printed_bills/',
                         train_data_dir='/',
                         img_mask_pair_file='/home/ydalal/rasidrev/resources/all_bills_labels.txt',
                         input_shape=input_shape,
                         train_split=0.8,
                         isAbsolutePath=True
                        )

### Miscelleneous Load Bill Data Here

In [ ]:
# Load Data
from glob import glob
import pandas as pd

data_dir='../../resources/train_data_photos/img/'
mask_dir='../../resources/train_data_photos/mask/'

df=pd.DataFrame(columns=['class_id','img','mask'],index=None)

fileList=glob(data_dir+'*.jpg')+glob(data_dir+'*.jpeg')+glob(data_dir+'*.tif')+glob(data_dir+'*.png')

counter=0
for fpath in fileList:
    fname=fpath.split("/")[-1]
    
    df=df.append(
        pd.DataFrame(
            [['0',data_dir+fname,mask_dir+fname]],
            columns=['class_id','img','mask'],
            index=None)
        )
    counter+=1
df=df.reset_index()
# df=df.drop(['index'],axis=1)

print('files read ... ',counter)

print(df[0:1]['class_id'][0],
      df[0:1]['img'][0],
      df[0:1]['mask'][0])